In [33]:
from hydradx.model.processing import get_omnipool_data, get_binance_orderbook
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.centralized_market import CentralizedMarket
from hydradx.model.amm.agents import Agent


from pprint import pprint

from hydradxapi import HydraDX


In [34]:

RPC="wss://rpc.hydradx.cloud"
            
asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, archive=False)



In [35]:
binance_ob = get_binance_orderbook(("DOT", "USDT"), archive=False)
pprint(binance_ob)
cex = CentralizedMarket(
    order_book={('DOT', 'USDT'): binance_ob},
    asset_list=["DOT", "USDT"]
)

OrderBook(bids=SortedList([[6.721, 737.16], [6.72, 1126.65], [6.719, 576.37], [6.718, 2154.57], [6.717, 2919.88], [6.716, 2167.22], [6.715, 2216.47], [6.714, 7141.17], [6.713, 1082.04], [6.712, 1408.45], [6.711, 3885.65], [6.71, 1072.23], [6.709, 2500.47], [6.708, 6917.97], [6.707, 2889.95], [6.706, 2822.65], [6.705, 2146.52], [6.704, 2615.42], [6.703, 9520.52], [6.702, 640.44], [6.701, 454.05], [6.7, 4491.26], [6.699, 1377.04], [6.698, 3.17], [6.697, 11.23], [6.696, 5312.36], [6.695, 50.77], [6.694, 1460.37], [6.693, 20.52], [6.692, 7.06], [6.691, 12.85], [6.69, 19.87], [6.688, 311.41], [6.687, 285.68], [6.686, 4.44], [6.685, 27.87], [6.684, 14.87], [6.682, 5.89], [6.681, 464.22], [6.68, 49.15], [6.679, 2.62], [6.678, 12.68], [6.677, 2796.73], [6.676, 11.15], [6.675, 39.19], [6.674, 56.03], [6.673, 260.23], [6.672, 9.89], [6.671, 2.32], [6.67, 78.83], [6.669, 4.54], [6.668, 205.32], [6.667, 1164.66], [6.666, 75.6], [6.664, 32.48], [6.662, 6.23], [6.661, 4.24], [6.66, 82.78], [6.659, 7

In [36]:
cex_spot = (cex.order_book[('DOT', 'USDT')].bids[0][0] + cex.order_book[('DOT', 'USDT')].asks[0][0]) / 2


usdt_data = tokens["USDT"]
usd_scale = 5
overall_scale = 1
usdt_lrna = usdt_data['LRNA'] * usd_scale * overall_scale
dot_liquidity = tokens['DOT']['liquidity'] * overall_scale
dot_lrna = tokens['DOT']['LRNA'] * overall_scale
usdt_liquidity = cex_spot * usdt_lrna * dot_liquidity / dot_lrna
usdt_data_up = {'LRNA': usdt_lrna, 'liquidity': usdt_liquidity}
hdx_data = {'LRNA': tokens['HDX']['LRNA'] * overall_scale, 'liquidity': tokens['HDX']['liquidity'] * overall_scale}
dot_data = {'LRNA': dot_lrna, 'liquidity': dot_liquidity}

op_state = OmnipoolState(
    tokens={"USDT": usdt_data_up, "DOT": dot_data, "HDX": hdx_data},
    preferred_stablecoin="USDT",
)

    
pprint(op_state)

Omnipool: omnipool
********************************
tvl cap: inf
lrna fee:

    USDT: 0%
    DOT: 0%
    HDX: 0%

asset fee:

    USDT: 0%
    DOT: 0%
    HDX: 0%

asset pools: (

    *USDT*
    asset quantity: 12409270.495458392
    lrna quantity: 384617.7888660239
    USD price: 1.0
    tvl: $12409270.495458392
    weight: 384617.7888660239/491586.48339881696 (0.7824010664548502)
    weight cap: 1
    total shares: 12409270.495458392
    protocol shares: 12409270.495458392

    *DOT*
    asset quantity: 410871.8244283416
    lrna quantity: 85596.43535107125
    USD price: 6.7215
    tvl: $2761674.9678950985
    weight: 85596.43535107125/491586.48339881696 (0.17412284153799265)
    weight cap: 1
    total shares: 410871.8244283416
    protocol shares: 410871.8244283416

    *HDX*
    asset quantity: 106629636.37322374
    lrna quantity: 21372.25918172182
    USD price: 0.006466799389
    tvl: $689552.4673129818
    weight: 21372.25918172182/491586.48339881696 (0.0434760920071572)
    

In [42]:
op_spot = op_state.price(op_state, "DOT", "USDT")
pprint(op_spot)
pprint(cex_spot)
pprint((cex.order_book[('DOT', 'USDT')].bids[0][0],cex.order_book[('DOT', 'USDT')].asks[0][0]))
pprint((cex.order_book[('DOT', 'USDT')].asks[0][0] - cex.order_book[('DOT', 'USDT')].bids[0][0])/cex.order_book[('DOT', 'USDT')].bids[0][0])

6.721500000000001
6.721500000000001
(6.721, 6.722)
0.0001487873828299857


In [43]:
agent = Agent(holdings={"USDT": 10000000, "DOT": 10000000})
trade_size_list = [10, 100, 1000]
dot_out_cex = []
dot_out_dex = []
for trade_size in trade_size_list:
    temp_op_state = op_state.copy()
    temp_cex_state = cex.copy()
    temp_agent1 = agent.copy()
    temp_agent2 = agent.copy()
    temp_op_state.swap(temp_agent1, tkn_buy="DOT", tkn_sell="USDT", sell_quantity=trade_size)
    temp_cex_state.swap(temp_agent2, tkn_buy="DOT", tkn_sell="USDT", sell_quantity=trade_size)
    dot_out_dex.append(temp_agent1.holdings["DOT"] - agent.holdings["DOT"])
    dot_out_cex.append(temp_agent2.holdings["DOT"] - agent.holdings["DOT"])
    pprint(trade_size)
    pprint(temp_op_state.price(temp_op_state, "DOT", "USDT"))
    
pprint("DEX")
pprint(dot_out_dex)
pprint("CEX")
pprint(dot_out_cex)

10
6.721559510142779
100
6.7220951132826405
1000
6.72745231831225
'DEX'
[1.4877565614879131, 14.87697290070355, 148.71048305742443]
'CEX'
[1.4876524843275547, 14.876524843275547, 148.7652484383434]


In [39]:
pprint(temp_op_state)

Omnipool: omnipool
********************************
tvl cap: inf
lrna fee:

    USDT: 0%
    DOT: 0%
    HDX: 0%

asset fee:

    USDT: 0%
    DOT: 0%
    HDX: 0%

asset pools: (

    *USDT*
    asset quantity: 12410270.495458392
    lrna quantity: 384586.7969719305
    USD price: 1.0
    tvl: $12410270.495458392
    weight: 384586.7969719305/491586.483398817 (0.782338021812371)
    weight cap: 1
    total shares: 12409270.495458392
    protocol shares: 12409270.495458392

    *DOT*
    asset quantity: 410723.1139452834
    lrna quantity: 85627.42724516468
    USD price: 6.727452318312
    tvl: $2763120.165095623
    weight: 85627.42724516468/491586.483398817 (0.1741858861804717)
    weight cap: 1
    total shares: 410871.8244283416
    protocol shares: 410871.8244283416

    *HDX*
    asset quantity: 106629636.37322374
    lrna quantity: 21372.25918172182
    USD price: 0.006467841684
    tvl: $689663.6068439202
    weight: 21372.25918172182/491586.483398817 (0.04347609200715719)
    